In [1]:
# Import PyTorch
import torch

# Import os for file related activities and folder navigation
import os

# Import Pandas for data manipulation
import pandas as pd

# Import the Python Imaging Library (PIL) for image processing
from PIL import Image

# Import PyTorch torchvision.transforms for image transformations during training
import torchvision.transforms as transforms

# Import PyTorch torchvision.datasets and models
from torchvision import datasets, models

# Import NumPy for numerical operations
import numpy as np

import json

# Import the requests library for HTTP requests
import requests

import warnings
warnings.filterwarnings('ignore')

# Import PyTorch Dataset and DataLoader for data handling
from torch.utils.data import Dataset, DataLoader

# Import PyTorch functional and neural network libraries
import torch.nn.functional as F
import torch.nn as nn

# Import torchvision for additional image-related functionality
import torchvision

# Import SummaryWriter from torch.utils.tensorboard for TensorBoard logging
from torch.utils.tensorboard import SummaryWriter

# Import datetime and time for date and time operations
import datetime
import time

# Import specific components from torchvision.models.resnet
from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck

# Clear GPU cache using torch.cuda.empty_cache()
torch.cuda.empty_cache()

# Determine the device (GPU or CPU) available for computation - ensure that cuda is displayed
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [3]:
## start the tensorboard
%load_ext tensorboard

In [4]:
%tensorboard --logdir {logs_base_dir}  --host localhost

In [6]:
## Instantiate the writer object that is called during model training
writer = SummaryWriter(log_dir='logs')

In [8]:

## Removal of erroneous mac os related artifacts

def remove_ds_store(root_dir):
    """
    Remove .DS_Store within a specified root dir and its subsiduaries
    
    A quirk of the mac os is the creation of .DS_Store file which needs to be removed from certain dirs
    Add the root directory and the function will walk through the sub dirs and remove the .DS_Store file
    
    Parameters
    ----------
    root_dir : ste
        The root directory to search for .DS_Store hidden files
    """
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file == ".DS_Store":
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Removed: {file_path}")

directory_to_clean = 'aicore_images_split'
remove_ds_store(directory_to_clean)


'\nimport os\n\ndef remove_ds_store(root_dir):\n    for root, _, files in os.walk(root_dir):\n        for file in files:\n            if file == ".DS_Store":\n                file_path = os.path.join(root, file)\n                os.remove(file_path)\n                print(f"Removed: {file_path}")\n\n# Replace \'path_to_your_directory\' with the actual path to the directory you want to clean\ndirectory_to_clean = \'aicore_images_split\'\nremove_ds_store(directory_to_clean)\n'

In [9]:

from PIL import Image

def resize_and_convert_to_rgb(image_path, target_size=(64, 64)):
    """
    A function to take a specified image and resizes it to 64x64 px
    
    Parameters
    ----------
    
    image_path : str
        a single file path of the target image
        
    Returns
    -------
    None
    
    """
    try:
        image = Image.open(image_path)
        image = image.resize(target_size)

        if image.mode != 'RGB':
            image = image.convert('RGB')

        image.save(image_path)
        #print(f"Processed: {image_path}")
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

def process_subdirectories(root_dir):
    """
    A function that works in tandem with the resize and convert to rgb function
    Specify a root directory it walks through each subn directory and if a file extn is of RGB type, it will leverage the resize
    and conversion function
    
    Parameters
    ----------
    
    root_dir : str
        a root direction containing the image subdirectories
    """
    for root, _, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                image_path = os.path.join(root, file)

                resize_and_convert_to_rgb(image_path)

root_directory = "aicore_images_split"
process_subdirectories(root_directory)


'\nfrom PIL import Image\n\ndef resize_and_convert_to_rgb(image_path, target_size=(64, 64)):\n    try:\n        image = Image.open(image_path)\n        image = image.resize(target_size)\n\n        if image.mode != \'RGB\':\n            image = image.convert(\'RGB\')\n\n        image.save(image_path)\n        #print(f"Processed: {image_path}")\n    except Exception as e:\n        print(f"Error processing {image_path}: {e}")\n\ndef process_subdirectories(root_dir):\n    for root, _, files in os.walk(root_dir):\n        for file in files:\n            if file.lower().endswith((\'.jpg\', \'.jpeg\', \'.png\', \'.bmp\', \'.gif\')):\n                image_path = os.path.join(root, file)\n\n                resize_and_convert_to_rgb(image_path)\n\nroot_directory = "aicore_images_split"\nprocess_subdirectories(root_directory)\n\n'

In [10]:
class ItemsTrainDataSet(Dataset):
    """
    A custom dataset class for image training data.
    
    This class inherits from the PyTorch Dataset class and will 
    load and preprocess image training data.

    Attributes
    ----------
    None

    Methods
    -------
    __init__():
        Initializes a new ItemsTrainDataSet instance.
    _load_examples():
        Loads and prepares a list of image examples.
    __getitem__(idx):
        Retrieves an image and its associated class label by index.
    __len__():
        Returns the total number of examples in the dataset.
    """
    def __init__(self):
        """
        Initialize a new ItemsTrainDataSet instance.

        This constructor sets up the necessary transformations for image preprocessing.
        """
        super().__init__()
        self.examples = self._load_examples()
        self.randomhflip = transforms.RandomHorizontalFlip()
        self.pil_to_tensor = transforms.ToTensor()
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )

    def _load_examples(self):
        """
        Load and prepare a list of image examples.

        This method scans the training directory, assigns class labels,
        and collects a list of image file paths.

        Returns
        -------
        list
            A list of tuples containing image file paths and their class labels.
        """
        class_names = os.listdir('aicore_images_split/train')
        class_encoder = {class_name: idx for idx, class_name in enumerate(class_names)}

        examples_list = []
        for cl_name in class_names:
            example_fp = os.listdir(os.path.join('aicore_images_split/train', cl_name))
            example_fp = [os.path.join('aicore_images_split/train', cl_name, img_name) for img_name in example_fp]
            example = [(img_name, class_encoder[cl_name]) for img_name in example_fp]
            examples_list.extend(example)

        return examples_list

    def __getitem__(self, idx):
        """
        Retrieve an image and its associated class label by index.

        Parameters
        ----------
        idx : int
            The index of the item to retrieve.

        Returns
        -------
        tuple
            A tuple containing the preprocessed image and its class label.
        """
        img_fp, img_class = self.examples[idx]
        img = Image.open(img_fp)
        img = self.randomhflip(img)
        features = self.pil_to_tensor(img)
        features = self.normalize(features)

        return features, img_class

    def __len__(self):
        """
        Return the total number of examples in the dataset.

        Returns
        -------
        int
            The total number of examples in the dataset.
        """
        return len(self.examples)

In [11]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class ItemsValDataSet(Dataset):
    """
    A custom dataset class for image validation data.
    
    This class inherits from the PyTorch Dataset class and is designed
    for loading and preprocessing image validation data.

    Attributes
    ----------
    None

    Methods
    -------
    __init__():
        Initializes a new ItemsValDataSet instance.
    _load_examples():
        Loads and prepares a list of image examples.
    __getitem__(idx):
        Retrieves an image and its corresponding class label by index.
    __len__():
        Returns the total number of examples in the dataset.
    """
    def __init__(self):
        """
        Initialize a new ItemsValDataSet instance.

        This constructor sets up the necessary transformations for image preprocessing.
        """
        super().__init__()
        self.examples = self._load_examples()
        self.pil_to_tensor = transforms.ToTensor()
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )

    def _load_examples(self):
        """
        Load and prepare a list of image examples.

        This method scans the validation directory, assigns class labels,
        and collects a list of image file paths.

        Returns
        -------
        list
            A list of tuples containing image file paths and their class labels.
        """
        class_names = os.listdir('aicore_images_split/val')
        class_encoder = {class_name: idx for idx, class_name in enumerate(class_names)}

        examples_list = []
        for cl_name in class_names:
            example_fp = os.listdir(os.path.join('aicore_images_split/val', cl_name))
            example_fp = [os.path.join('aicore_images_split/val', cl_name, img_name) for img_name in example_fp]
            example = [(img_name, class_encoder[cl_name]) for img_name in example_fp]
            examples_list.extend(example)

        return examples_list

    def __getitem__(self, idx):
        """
        Retrieve an image and its corresponding class label by index.

        Parameters
        ----------
        idx : int
            The index of the item to retrieve.

        Returns
        -------
        tuple
            A tuple containing the preprocessed image and its class label.
        """
        img_fp, img_class = self.examples[idx]
        img = Image.open(img_fp)
        features = self.pil_to_tensor(img)
        features = self.normalize(features)

        return features, img_class

    def __len__(self):
        """
        Return the total number of examples in the dataset.

        Returns
        -------
        int
            The total number of examples in the dataset.
        """
        return len(self.examples)

In [12]:
# Create the traindataset object 
traindataset = ItemsTrainDataSet()
# Check the number of training images is as expected
len(traindataset)

8816

In [13]:
# Create the validation dataset object
valdataset = ItemsValDataSet()
# Check the number of training images is as expected
len(valdataset)

3788

In [14]:
## Created a classifier based on the RESNET50 pretrained model

class ItemClassifier(torch.nn.Module):
    """
    A custom nn.Module class housing the classifier which is
    based on a gpu-derived pretrained resnet50 from NVIDA torchhub
    
    Attributes
    ----------
    None
    
    
    Methods
    -------
    __init__():
        Initialises the classifier and loads the model from the torchhub
        unless it is able to detect a cached instance.
        It adapts the first layer to handle 64 x 64 images
        Replaces the final layer with a 13 class output
    
    forward():
        Initiates the forward pass
        
    """
    def __init__(self):
        super().__init__()
        self.resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        self.resnet50.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet50.avgpool = nn.AdaptiveAvgPool2d(1)
        #self.resnet50 = model
        self.resnet50.fc = torch.nn.Linear(2048,13)

    def forward(self, X):
        return F.softmax(self.resnet50(X))

In [15]:
## Create the classifier object from the ItemClassifier class
classifier = ItemClassifier()

Using cache found in /home/ubuntu/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [17]:
## define the layers to unfreeze and then retrain
layers_to_unfreeze = ['layers.3']

for name, param in classifier.resnet50.named_parameters():
    for layer_name in layers_to_unfreeze:
        if layer_name in name:
            param.requires_grad = True
            break


In [19]:


def train(model, traindataloader, valdataloader, epochs):
    """
    Train the modified pretrained resnet50 classifier.
    
    The model weights are written after each epoch to model_evaluation in
    the current working directory
    
    In addition at the end of each epoch:
        Validation Accuracy
        Validation Loss
        Training Accuracy
        Training Loss
    are written to the logs file for the tensorboard to visualise
    performance

    Parameters
    ----------
    model : torch.nn.Module
        The deep learning model to be trained.
    traindataloader : torch.utils.data.DataLoader
        DataLoader for the training dataset.
    valdataloader : torch.utils.data.DataLoader
        DataLoader for the validation dataset.
    epochs : int
        The number of training epochs.

    Returns
    -------
    None
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    writer = SummaryWriter()
    #optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
    
    model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    model_path = os.path.join('model_evaluation', time.strftime("%Y%m%d-%H%M%S"))
    os.makedirs(model_path)
    os.makedirs(os.path.join(model_path, 'weights'))
    global_step = 0

    for epoch in range(epochs):
        training_loss = 0.0
        validation_loss = 0.0
        tr_num_correct = 0
        tr_num_examples = 0
        epoch_combo = 'epoch' + str(epoch)
        os.makedirs(os.path.join(model_path, 'weights', epoch_combo))

        model.train()
        for inputs, labels in traindataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            predictions = model(inputs)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()
            
            training_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(predictions, dim=1), dim=1)[1], labels)
            tr_num_correct += torch.sum(correct).item()
            tr_num_examples += correct.shape[0]

        training_loss /= len(traindataloader.dataset)
        training_accuracy = tr_num_correct / tr_num_examples
        writer.add_scalar('Training Loss', training_loss, global_step)
        writer.add_scalar('Training Accuracy', training_accuracy, global_step)

        model.eval()
        val_num_correct = 0
        val_num_examples = 0
        with torch.no_grad():
            for inputs, labels in valdataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                predictions = model(inputs)
                loss = criterion(predictions, labels)
                validation_loss += loss.item() * inputs.size(0)
                correct = torch.eq(torch.max(F.softmax(predictions, dim=1), dim=1)[1], labels)
                val_num_correct += torch.sum(correct).item()
                val_num_examples += correct.shape[0]

        validation_loss /= len(valdataloader.dataset)
        validation_accuracy = val_num_correct / val_num_examples
        writer.add_scalar('Validation Loss', validation_loss, global_step)
        writer.add_scalar('Validation Accuracy', validation_accuracy, global_step)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, train_accuracy = {:.2f}, val_accuracy = {:.2f}'.format(epoch, training_loss, validation_loss, training_accuracy, validation_accuracy))

        # Save the model checkpoint at the end of each epoch
        model_save_dir = os.path.join(model_path, 'weights', epoch_combo, 'weights.pt')
        torch.save({'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict()},
                    model_save_dir)

        global_step += 1

    writer.close()


In [ ]:
## Create the train and validation loaders
## Pass the loaders, classifier and desired number of epochs to the train function define above
train_loader = DataLoader(dataset = traindataset, batch_size=32)
val_loader = DataLoader(dataset = valdataset, batch_size=32)
train(classifier, traindataloader= train_loader, valdataloader= val_loader, epochs=500)


Epoch: 0, Training Loss: 2.57, Validation Loss: 2.56, train_accuracy = 0.08, val_accuracy = 0.09
Epoch: 1, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.09, val_accuracy = 0.08
Epoch: 2, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.09, val_accuracy = 0.09
Epoch: 3, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.09, val_accuracy = 0.09
Epoch: 4, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.09, val_accuracy = 0.09
Epoch: 5, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.10, val_accuracy = 0.10
Epoch: 6, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.10, val_accuracy = 0.09
Epoch: 7, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.10, val_accuracy = 0.09
Epoch: 8, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.11, val_accuracy = 0.09
Epoch: 9, Training Loss: 2.56, Validation Loss: 2.56, train_accuracy = 0.11, val_accuracy = 0.09
Epoch: 10, Training Loss: 2.56